In [73]:
import backtrader as bt
import yfinance as yf
import math
import talib as ta
%matplotlib inline

In [74]:
df = yf.download('AAPL', start='2023-01-01', end='2023-12-31')
df.columns = [col[0] for col in df.columns] # Flatten multi-level column index
print(df.info())
df.head()

[*********************100%***********************]  1 of 1 completed

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 250 entries, 2023-01-03 00:00:00+00:00 to 2023-12-29 00:00:00+00:00
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Adj Close  250 non-null    float64
 1   Close      250 non-null    float64
 2   High       250 non-null    float64
 3   Low        250 non-null    float64
 4   Open       250 non-null    float64
 5   Volume     250 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 13.7 KB
None


,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2023-01-03 00:00:00+00:00,123.904625,125.070000,130.899994,124.169998,130.279999,112117500
2023-01-04 00:00:00+00:00,125.182617,126.360001,128.660004,125.080002,126.889999,89113600
2023-01-05 00:00:00+00:00,123.855095,125.019997,127.769997,124.760002,127.129997,80962700
2023-01-06 00:00:00+00:00,128.412231,129.619995,130.289993,124.889999,126.010002,87754700
2023-01-09 00:00:00+00:00,128.937286,130.149994,133.410004,129.889999,130.470001,70790800


In [75]:
data = bt.feeds.PandasData(dataname=df)
data

In [76]:
class SMA_Cross(bt.Strategy):
    
    params = dict(
        sma_period_short = 5,
        sma_period_long = 10
    )

    def __init__(self):

        self.sma_short = bt.indicators.SMA(self.data.close, period=self.params.sma_period_short)
        self.sma_long = bt.indicators.SMA(self.data.close, period=self.params.sma_period_long)

        self.crossover = bt.indicators.CrossOver(self.sma_short, self.sma_long)

        self.dataclose = self.datas[0].close
    
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.datetime().date()
        print('%s, %s' % (dt.isoformat(), txt))
    
    def notify_order(self, order):
        if order.status in [order.Submitted]:
            self.log('Order Submitted')
            pass

        if order.status in [order.Accepted]:
            pass
        
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm: %.2f' % (order.executed.price, order.executed.value, order.executed.comm))
            else:
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm: %.2f' % (order.executed.price, order.executed.value, order.executed.comm))
        
        if order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
            
        self.order = None

    def next(self):

        if self.crossover > 0: # buy in
            self.order = self.buy()
            pass
        elif self.crossover < 0 and self.position: # sell out
            self.order = self.sell()
            pass

    def stop(self):
        pass

In [ ]:
cerebro = bt.Cerebro()

cerebro.adddata(data)
cerebro.addstrategy(SMA_Cross)
cerebro.broker.setcash(10000)
cerebro.broker.setcommission(commission=0.001)

# cerebro.optstrategy(
#     SMA_Cross,
#     sma_period_long = range(10, 30)
# )

cerebro.run(maxcpus = 1)
print(f'Final Portfolio Value: {cerebro.broker.getvalue()}')

cerebro.plot(iplot=False)
fig = cerebro.plot(iplot=False)[0][0]
fig.savefig("backtest_plot.png")

2023-03-07, Order Submitted
2023-03-07, BUY EXECUTED, Price: 153.70, Cost: 153.70, Comm: 0.15
2023-03-16, Order Submitted
2023-03-16, SELL EXECUTED, Price: 152.16, Cost: 153.70, Comm: 0.15
2023-03-17, Order Submitted
2023-03-17, BUY EXECUTED, Price: 156.08, Cost: 156.08, Comm: 0.16
2023-04-13, Order Submitted
2023-04-13, SELL EXECUTED, Price: 161.63, Cost: 156.08, Comm: 0.16
2023-04-19, Order Submitted
2023-04-19, BUY EXECUTED, Price: 165.80, Cost: 165.80, Comm: 0.17
2023-04-27, Order Submitted
2023-04-27, SELL EXECUTED, Price: 165.19, Cost: 165.80, Comm: 0.17
2023-05-02, Order Submitted
2023-05-02, BUY EXECUTED, Price: 170.09, Cost: 170.09, Comm: 0.17
2023-05-19, Order Submitted
2023-05-19, SELL EXECUTED, Price: 176.39, Cost: 170.09, Comm: 0.18
2023-05-22, Order Submitted
2023-05-22, BUY EXECUTED, Price: 173.98, Cost: 173.98, Comm: 0.17
2023-05-30, Order Submitted
2023-05-30, SELL EXECUTED, Price: 176.96, Cost: 173.98, Comm: 0.18
2023-06-01, Order Submitted
2023-06-01, BUY EXECUTED, P

: 